In [2]:
import pandas as pd 
import torch
import torch.nn as nn 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.utils.data import DataLoader, TensorDataset


In [13]:
data = pd.read_csv("./SKN07-2nd-2Team/SKN07-2nd-2Team/df_encoding.csv")
data = pd.DataFrame(data)
data.drop(columns='Unnamed: 0', inplace=True)

In [22]:
X.shape

(49752, 56)

In [16]:
X = data.drop(columns=['Churn']).values
y = data['Churn'].values

data 에서 column 들 중 X, Churn을 y 로 둬....? 

In [17]:
# 데이터 정규화 (특징들을 스케일링)
scaler = StandardScaler()  # 데이터 정규화를 위한 StandardScaler 사용
data = scaler.fit_transform(data)  # 데이터를 평균 0, 표준편차 1로 변환

# 학습 데이터(60%), 나머지 데이터(40%) 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
# 테스트 데이터(20%)와 검증 데이터(20%) 나누기
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 데이터를 PyTorch 텐서 형식으로 변환
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)  # 훈련 데이터
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  # 타겟 데이터 (이진 분류이므로 2D 텐서로 변환)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)  # 테스트 데이터
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)  # 타겟 데이터(이진 분류이므로 2D 텐서로 변환)
X_val_tensor = torch.tensor(X_train, dtype=torch.float32) # 검증 데이터터
y_val_tensor = torch.tensor(X_train, dtype=torch.float32).view(-1, 1)  # 타겟 데이터(이진 분류이므로 2D 텐서로 변환)


# DataLoader로 배치 처리 (훈련 데이터, 테스트 데이터)
train_data = TensorDataset(X_train_tensor, y_train_tensor)  # 훈련 데이터셋 생성
test_data = TensorDataset(X_test_tensor, y_test_tensor)  # 테스트 데이터셋 생성

train_dl = DataLoader(train_data, batch_size=64, shuffle=True)  # 훈련 데이터셋에 배치 크기 설정, 데이터를 섞음
test_dl = DataLoader(test_data, batch_size=64, shuffle=False)  # 테스트 데이터셋에 배치 크기 설정

In [23]:
# 신경망 모델 정의
class tele_churn_predict(nn.Module):
    def __init__(self):
        super(tele_churn_predict, self).__init__()
        # 개의 입력 특징을 받아들이는 첫 번째 Fully Connected Layer
        self.fc1 = nn.Linear(56, 32)  # 개의 입력 특징
        self.fc2 = nn.Linear(32, 16)  # 32개의 출력에서 16개로 변환
        self.fc3 = nn.Linear(16, 1)  # 16개의 출력에서 1개의 생존 확률 값으로 변환
        self.sigmoid = nn.Sigmoid()  # 생존 확률을 0과 1 사이로 변환하기 위한 Sigmoid 함수

    def forward(self, x):
        x = nn.ReLU()(self.fc1(x))  # 첫 번째 레이어 후 ReLU 활성화 함수
        x = nn.ReLU()(self.fc2(x))  # 두 번째 레이어 후 ReLU 활성화 함수
        x = self.fc3(x)  # 세 번째 레이어
        x = self.sigmoid(x)  # 출력값을 확률로 변환
        return x

In [24]:
# 모델 인스턴스화
model = tele_churn_predict()
# 손실 함수와 옵티마이저 설정
loss_fn = nn.BCELoss()  # 이진 분류 문제이므로 Binary Cross Entropy 손실 함수 사용
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adam 옵티마이저 사용

In [25]:
# 훈련 루프
num_epochs = 20  # 훈련할 에포크 수 설정
for epoch in range(num_epochs):
    model.train()  # 모델을 훈련 모드로 설정
    total_loss = 0  # 손실을 누적할 변수
    correct = 0  # 정확도를 계산할 변수
    
    for X_batch, y_batch in train_dl:
        optimizer.zero_grad()  # 기울기 초기화
        output = model(X_batch)  # 순전파: 배치 데이터를 모델에 통과시켜 예측값 생성
        loss = loss_fn(output, y_batch)  # 손실 계산
        loss.backward()  # 역전파
        optimizer.step()  # 옵티마이저로 파라미터 업데이트
        
        total_loss += loss.item()  # 손실값 누적
        predicted = (output > 0.5).float()  # 예측값이 0.5보다 크면 생존, 작으면 사망
 
        correct += (predicted == y_batch).sum().item()  # 정확도 계산
 # 에포크마다 손실과 정확도 출력
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_dl):.4f}, Accuracy: {correct/len(train_dl.dataset):.4f}")

Epoch 1/20, Loss: 0.6220, Accuracy: 0.7043
Epoch 2/20, Loss: 0.5932, Accuracy: 0.7132
Epoch 3/20, Loss: 0.5886, Accuracy: 0.7140
Epoch 4/20, Loss: 0.5866, Accuracy: 0.7151
Epoch 5/20, Loss: 0.5841, Accuracy: 0.7162
Epoch 6/20, Loss: 0.5827, Accuracy: 0.7162
Epoch 7/20, Loss: 0.5819, Accuracy: 0.7172
Epoch 8/20, Loss: 0.5808, Accuracy: 0.7172
Epoch 9/20, Loss: 0.5805, Accuracy: 0.7180
Epoch 10/20, Loss: 0.5789, Accuracy: 0.7182
Epoch 11/20, Loss: 0.5788, Accuracy: 0.7176
Epoch 12/20, Loss: 0.5785, Accuracy: 0.7193
Epoch 13/20, Loss: 0.5772, Accuracy: 0.7192
Epoch 14/20, Loss: 0.5774, Accuracy: 0.7195
Epoch 15/20, Loss: 0.5763, Accuracy: 0.7198
Epoch 16/20, Loss: 0.5753, Accuracy: 0.7201
Epoch 17/20, Loss: 0.5753, Accuracy: 0.7211
Epoch 18/20, Loss: 0.5755, Accuracy: 0.7192
Epoch 19/20, Loss: 0.5734, Accuracy: 0.7208
Epoch 20/20, Loss: 0.5737, Accuracy: 0.7218
